<a href="https://colab.research.google.com/github/hyeonsang010716/Recommendation-system/blob/main/bayesian_personalized_ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#CSR 매트릭스 표현하기 중간고사 + 복잡도 O(N +X)

In [ ]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 18.4 MB/s eta 0:00:00


In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

--2024-04-03 04:54:17--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  4.60MB/s    in 0.2s    

2024-04-03 04:54:17 (4.60 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
from scipy.sparse import csr_matrix
import numpy as np
rows = [] #유저가 로우
cols = [] #아이템이 컬럼
data = []
with open("ml-latest-small/ratings.csv" , "r") as f:
  f.readline()
  for line in f:
    uid , mid , _ , _ = line.split(",")
    rows.append(int(uid))
    cols.append(int(mid))
    data.append(1)

rows = np.array(rows)
cols = np.array(cols)
data = np.array(data)

ratings_csr = csr_matrix((data , (rows , cols)))


In [ ]:
ratings_csr

<611x193610 sparse matrix of type '<class 'numpy.int64'>'
	with 100836 stored elements in Compressed Sparse Row format>

In [ ]:
from implicit import bpr

model = bpr.BayesianPersonalizedRanking(factors = 10)
model.fit(ratings_csr)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
users = [2,3,4,5] # 2번 3번 4번 5번 유저들 추천
ids , scores = model.recommend(users , ratings_csr[users])
print(ids)
print(scores)

#[[115569 112556 122904 106920  89745  84152 112852  68954 139385 112138] - 2번 유저한테 추천 무비 아이디
# [  1269   1295   3706   2150   2384   1959   2243   1077   6662   2352] - 3번
# [  2396   1285   2352   2248   2289   2144   1958   1247   1296    838] - 4번
# [    17    539    339     62    454    161    141    292    225    587] - 5번 ]

[[115569 112556 122904 106920  89745  84152 112852  68954 139385 112138]
 [  1269   1295   3706   2150   2384   1959   2243   1077   6662   2352]
 [  2396   1285   2352   2248   2289   2144   1958   1247   1296    838]
 [    17    539    339     62    454    161    141    292    225    587]]
[[3.3710876 3.358287  3.2743332 3.2371976 3.2007866 3.1546407 3.148889
  3.144515  3.135222  3.123825 ]
 [2.5056498 2.3934789 2.3095949 2.258928  2.24599   2.2383065 2.214796
  2.1689749 2.1677089 2.1525428]
 [4.2362504 4.0210643 3.7800968 3.7768152 3.6185904 3.573056  3.5612266
  3.5382156 3.4476848 3.411141 ]
 [4.4059424 4.3746977 4.3660607 4.201074  4.166909  4.088395  4.0699553
  4.0687838 4.037872  4.014175 ]]


In [ ]:
model.user_factors.shape # 611명 유저들에 factor들이고 우리가 factor를 10으로 설정해주었지만 내부적으로 우리가 설정 + 1 로 만들어줌

(611, 11)

In [ ]:
import torch

users = torch.from_numpy(rows)
items = torch.from_numpy(cols)
# numpy -> torch로 변환
n_items = max(items) + 1
n_users = max(users) + 1
n_factors = 10
item_bias = torch.randn(n_items)
item_factor = torch.randn(n_items , n_factors , requires_grad=True)

user_factor = torch.randn(n_users , n_factors , requires_grad=True)



In [ ]:
optim = torch.optim.Adam([item_bias , item_factor , user_factor] , lr = 0.1)
logsigmoid = torch.nn.LogSigmoid()
lmd = 0.01
for epoch in range(100):
  neg_items = torch.randint(1 , n_items , (len(items),))
  pos_pref = item_bias[items] + (user_factor[users] * item_factor[items]).sum(dim = 1) # 각 줄마다 sum
  neg_pref = item_bias[neg_items] + (user_factor[users] * item_factor[neg_items]).sum(dim = 1)
  reg = (item_bias **2).sum() + (user_factor ** 2).sum() + (item_factor ** 2).sum()
  cost = - logsigmoid(pos_pref - neg_pref).sum() - lmd * reg
  optim.zero_grad()
  cost.backward()
  optim.step()
  with torch.no_grad():
    tran_acc = (pos_pref > neg_pref).sum() / len(pos_pref)
    print("epoch : {} , tran acc : {}".format(epoch , tran_acc))



epoch : 0 , tran acc : 0.672755777835846
epoch : 1 , tran acc : 0.7273592948913574
epoch : 2 , tran acc : 0.7808818221092224
epoch : 3 , tran acc : 0.8317664265632629
epoch : 4 , tran acc : 0.8732396960258484
epoch : 5 , tran acc : 0.9040818810462952
epoch : 6 , tran acc : 0.9309670925140381
epoch : 7 , tran acc : 0.9477964043617249
epoch : 8 , tran acc : 0.9610456824302673
epoch : 9 , tran acc : 0.969752848148346
epoch : 10 , tran acc : 0.9754254221916199
epoch : 11 , tran acc : 0.9804930686950684
epoch : 12 , tran acc : 0.9834384322166443
epoch : 13 , tran acc : 0.9857788681983948
epoch : 14 , tran acc : 0.987266480922699
epoch : 15 , tran acc : 0.9882383346557617
epoch : 16 , tran acc : 0.9893391132354736
epoch : 17 , tran acc : 0.989785373210907
epoch : 18 , tran acc : 0.9901424050331116
epoch : 19 , tran acc : 0.9903605580329895
epoch : 20 , tran acc : 0.9907076954841614
epoch : 21 , tran acc : 0.9911043643951416
epoch : 22 , tran acc : 0.9916895031929016
epoch : 23 , tran acc : 0

In [ ]:
with torch.no_grad():
  n_outputs = 10
  uid = 2
  scores_all = item_bias + (user_factor[uid] * item_factor).sum(dim = 1)
  scores_all = scores_all.numpy()
  ids = np.argsort(scores_all)[::-1]
  pure_ids = ids[np.isin(ids , items[users==uid] , invert = True)]
  pure_ids = pure_ids[:n_outputs]
  scores = scores_all[pure_ids]
  print(pure_ids)
  print(scores)

[164216 183415 189192 187318 161225 157626 106503  87286 185783  30930]
[39.663452 38.32841  38.307327 38.012184 36.8266   36.72903  36.57448
 36.550068 36.44435  36.251198]
